In [1]:
import numpy as np 
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F

import albumentations
import torchvision
from torchvision import transforms

import timm
import tqdm

from src.losses import ArcMarginProduct
from src.utils.metrics import accuracy
from src.utils.data_loaders import ImagesDS
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
path_data = '/home/marchevskiy/Data/other'
device = 'cuda'
batch_size = 32

In [4]:
transforms_train = albumentations.Compose([
    albumentations.RandomCrop(height=334, width=334, p=1),
    albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.3),
    albumentations.RandomRotate90(p=0.3),
    albumentations.Flip(p=0.3),
    # albumentations.ElasticTransform(alpha=0.5, sigma=25, alpha_affine=25, p=0.3),
    # albumentations.RandomBrightness(limit=0.3, p=0.3),
    # albumentations.RandomContrast(limit=0.15, p=0.3),
    albumentations.Normalize(
     mean=[0.01639522, 0.06704897, 0.0366343 , 0.03555733, 0.02223296, 0.03011626],
     std=[0.01954379, 0.05231356, 0.01980171, 0.02657354, 0.01802711, 0.01548923])
])

transforms_test = albumentations.Compose([
    albumentations.RandomCrop(height=334, width=334, p=1),
    albumentations.Normalize(
     mean=[0.01639522, 0.06704897, 0.0366343 , 0.03555733, 0.02223296, 0.03011626],
     std=[0.01954379, 0.05231356, 0.01980171, 0.02657354, 0.01802711, 0.01548923])
])

In [5]:
ds = ImagesDS(pd.read_csv(path_data+'/train.csv'), path_data, transforms_train, mode='train')
loader = D.DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=8)

In [5]:
class ResNet50(nn.Module):
    def __init__(self, num_classes=1108):
        super().__init__()
        # preloaded = torchvision.models.resnet50(True)
        
        preloaded = timm.models.seresnext50_32x4d(True)
        tail = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        tail.weight.data[:, :3, :, :] = preloaded.layer0.conv1.weight.data
        tail.weight.data[:, 3:, :, :] = preloaded.layer0.conv1.weight.data
        preloaded.layer0.conv1 = tail
        
        self.features = nn.Sequential(*list(preloaded.children())[:-2])
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.bn1 = nn.BatchNorm1d(2048 * 2)
        self.fc1 = nn.Linear(2048 * 2, 512 * 2)
        self.bn2 = nn.BatchNorm1d(512 * 2)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(512 * 2, num_classes)
        # self.bn3 = nn.BatchNorm1d(512)
        del preloaded
        
    def forward(self, x):
        x = self.features(x)
        x = torch.cat((nn.AdaptiveAvgPool2d(1)(x), nn.AdaptiveMaxPool2d(1)(x)), dim=1)
        x = x.view(x.size(0), -1)
        x = self.bn1(x)
        x = F.dropout(x, p=0.25)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = F.dropout(x, p=0.5)

        x = x.view(x.size(0), -1)

        x = self.fc2(x)
        # feature = self.bn3(x)

        return x

In [6]:
model = ResNet50()
model.to(device);

In [9]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=30e-5)

epochs = 40
tlen = len(loader)
for epoch in range(epochs):
    tloss = 0
    acc = np.zeros(1)
    model.train()
    for x, y in tqdm.tqdm_notebook(loader):
        x = x.to(device)
        optimizer.zero_grad()
        output = model(x)
        target = torch.zeros_like(output, device=device)
        target[np.arange(x.size(0)), y] = 1
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        tloss += loss.item() 
        acc += accuracy(output.cpu(), y)
        del loss, output, y, x, target
    if epoch > 10:
        optimizer.param_groups[0]['lr'] = 15e-4
    if epoch > 20:
        optimizer.param_groups[0]['lr'] = 7.5e-4
    if epoch > 25:
        optimizer.param_groups[0]['lr'] = 3e-4
    if epoch > 35:
        optimizer.param_groups[0]['lr'] = 1e-4
    print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))


Epoch 1 -> Train Loss: 0.0074, ACC: 0.16%



Epoch 2 -> Train Loss: 0.0073, ACC: 0.17%



Epoch 3 -> Train Loss: 0.0073, ACC: 0.27%



Epoch 4 -> Train Loss: 0.0072, ACC: 0.34%



Epoch 5 -> Train Loss: 0.0071, ACC: 0.45%


...


Epoch 30 -> Train Loss: 0.0034, ACC: 41.86%



Epoch 31 -> Train Loss: 0.0034, ACC: 42.75%



Epoch 32 -> Train Loss: 0.0034, ACC: 43.60%



Epoch 33 -> Train Loss: 0.0033, ACC: 44.33%


RuntimeError: DataLoader worker (pid 18379) is killed by signal: Bus error. 

In [14]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=30e-4)

epochs = 2
tlen = len(loader)
for epoch in range(epochs):
    tloss = 0
    acc = np.zeros(1)
    model.train()
    for x, y in tqdm.tqdm_notebook(loader):
        x = x.to(device)
        optimizer.zero_grad()
        output = model(x)
        target = torch.zeros_like(output, device=device)
        target[np.arange(x.size(0)), y] = 1
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        tloss += loss.item() 
        acc += accuracy(output.cpu(), y)
        del loss, output, y, x, target
    print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))

Epoch 1 -> Train Loss: 0.0058, ACC: 11.74%


Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, b

KeyboardInterrupt: 

In [10]:
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=30e-5)

# epochs = 1
# tlen = len(loader)
# for epoch in range(epochs):
#     tloss = 0
#     acc = np.zeros(1)
#     model.train()
#     for x, y in tqdm.tqdm_notebook(loader):
#         x = x.to(device)
#         optimizer.zero_grad()
#         output = model(x)
#         # target = torch.zeros_like(output, device=device)
#         # target[np.arange(x.size(0)), y] = 1
#         target = y.to(device).long()
#         loss = criterion(output, target)
#         loss.backward()
#         optimizer.step()
#         tloss += loss.item() 
#         # acc += accuracy(output.cpu(), y)
#         output = output.data.cpu().numpy()
#         output = np.argmax(output, axis=1)
#         y = y.data.cpu().numpy()
#         acc += np.mean((output == y).astype(int))
#         del loss, output, y, x, target
#     print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))


Epoch 1 -> Train Loss: 6.7496, ACC: 0.01%


---

In [12]:
model.load_state_dict(torch.load('../../data/seresnext50_v1.pth'));

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [13]:
torch.save(model.state_dict(), '../../data/seresnext50_v1.pth')

In [7]:
n_folds = 4
train_df = pd.read_csv(path_data+'/train.csv')
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)
fold = folds.split(train_df, train_df['sirna'])

In [10]:
for fold_, (train_index, val_index) in enumerate(fold):
    ds_train = ImagesDS(train_df.iloc[train_index,:], path_data, transforms_train, mode='train')
    loader_train = D.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=8)
    ds_valid = ImagesDS(train_df.iloc[val_index,:], path_data, transforms_train, mode='train')
    loader_valid = D.DataLoader(ds_valid, batch_size=batch_size, shuffle=False, num_workers=8)
    
    model.load_state_dict(torch.load('../../data/seresnext50_v1.pth'))

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

    epochs = 20
    for epoch in range(epochs):
        tloss = 0
        acc = np.zeros(1)
        tlen = len(loader_train)
        model.train()
        for x, y in tqdm.tqdm_notebook(loader_train):
            x = x.to(device)
            optimizer.zero_grad()
            output = model(x)
            target = torch.zeros_like(output, device=device)
            target[np.arange(x.size(0)), y] = 1
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            tloss += loss.item() 
            acc += accuracy(output.cpu(), y)
            del loss, output, y, x, target
#         if epoch > 10:
#             optimizer.param_groups[0]['lr'] = 15e-4
#         if epoch > 20:
#             optimizer.param_groups[0]['lr'] = 7.5e-4
        if epoch > 25:
            optimizer.param_groups[0]['lr'] = 1e-4
        if epoch > 35:
            optimizer.param_groups[0]['lr'] = 1e-4
        print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))
        
        acc = np.zeros(1)
        tlen = len(loader_valid)
        model.eval()
        for x, y in loader_valid:
            x = x.to(device)
            output = model(x)
            target = torch.zeros_like(output, device=device)
            target[np.arange(x.size(0)), y] = 1
            acc += accuracy(output.cpu(), y)
            del output, y, x, target
        print('Epoch {} -> Valid ACC: {:.2f}%'.format(epoch+1, acc[0]/tlen))
        
    torch.save(model.state_dict(), '../../data/resnext50_v1(d05(384))_HEPG2_fold{}.pth'.format(fold_))

Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiproce

KeyboardInterrupt: 

---

In [6]:
train_df = pd.read_csv(path_data+'/train.csv')
train_df_HEPG2 = train_df[train_df['experiment'].isin(['HEPG2-01', 'HEPG2-02', 'HEPG2-03', 'HEPG2-04', 'HEPG2-05',
                                                       'HEPG2-06', 'HEPG2-07'])]

In [7]:
n_folds = 4
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)
fold = folds.split(train_df_HEPG2, train_df_HEPG2['sirna'])

In [8]:
model = ResNet50()
model.to(device);

In [9]:
for fold_, (train_index, val_index) in enumerate(fold):
    ds_train = ImagesDS(train_df_HEPG2.iloc[train_index,:], path_data, transforms_train, mode='train')
    loader_train = D.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=4)
    ds_valid = ImagesDS(train_df_HEPG2.iloc[val_index,:], path_data, transforms_train, mode='train')
    loader_valid = D.DataLoader(ds_valid, batch_size=batch_size, shuffle=False, num_workers=4)
    
    model.load_state_dict(torch.load('../../data/seresnext50_v1.pth'))

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=30e-5)

    epochs = 40
    for epoch in range(epochs):
        tloss = 0
        acc = np.zeros(1)
        tlen = len(loader_train)
        model.train()
        for x, y in tqdm.tqdm_notebook(loader_train):
            x = x.to(device)
            optimizer.zero_grad()
            output = model(x)
            target = torch.zeros_like(output, device=device)
            target[np.arange(x.size(0)), y] = 1
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            tloss += loss.item() 
            acc += accuracy(output.cpu(), y)
            del loss, output, y, x, target
        if epoch > 10:
            optimizer.param_groups[0]['lr'] = 15e-5
        if epoch > 20:
            optimizer.param_groups[0]['lr'] = 7.5e-5
        if epoch > 25:
            optimizer.param_groups[0]['lr'] = 3e-5
        if epoch > 35:
            optimizer.param_groups[0]['lr'] = 1e-5
        print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))
        
        acc = np.zeros(1)
        tlen = len(loader_valid)
        model.eval()
        for x, y in loader_valid:
            x = x.to(device)
            output = model(x)
            target = torch.zeros_like(output, device=device)
            target[np.arange(x.size(0)), y] = 1
            acc += accuracy(output.cpu(), y)
            del output, y, x, target
        print('Epoch {} -> Valid ACC: {:.2f}%'.format(epoch+1, acc[0]/tlen))
        
    torch.save(model.state_dict(), '../../data/seresnext50_v1_HEPG2_fold{}.pth'.format(fold_))


Epoch 1 -> Train Loss: 0.0065, ACC: 9.40%
Epoch 1 -> Valid ACC: 14.32%



Epoch 2 -> Train Loss: 0.0052, ACC: 17.38%
Epoch 2 -> Valid ACC: 18.86%



Epoch 3 -> Train Loss: 0.0048, ACC: 20.86%
Epoch 3 -> Valid ACC: 21.61%



Epoch 4 -> Train Loss: 0.0045, ACC: 24.65%
Epoch 4 -> Valid ACC: 23.67%



Epoch 5 -> Train Loss: 0.0043, ACC: 28.41%
Epoch 5 -> Valid ACC: 27.56%


...


Epoch 36 -> Train Loss: 0.0030, ACC: 50.61%
Epoch 36 -> Valid ACC: 34.24%



Epoch 37 -> Train Loss: 0.0030, ACC: 51.13%
Epoch 37 -> Valid ACC: 32.41%



Epoch 38 -> Train Loss: 0.0030, ACC: 51.48%
Epoch 38 -> Valid ACC: 36.16%



Epoch 39 -> Train Loss: 0.0030, ACC: 51.25%
Epoch 39 -> Valid ACC: 39.73%



Epoch 40 -> Train Loss: 0.0030, ACC: 51.33%
Epoch 40 -> Valid ACC: 41.52%
